In [1]:
import pyspark
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder.appName('Spam Classifier').getOrCreate()
cores = spark._jsc.sc().getExecutorMemoryStatus().keySet().size()
print(f'Cores: {cores}')
spark

Cores: 1


In [3]:
data = spark.read.csv('./data/SMSSpamCollection', inferSchema= True, sep= '\t')

In [5]:
data = data.withColumnRenamed('_c0', 'class').withColumnRenamed('_c1', 'text')

In [9]:
data.show(4, False)

+-----+-----------------------------------------------------------------------------------------------------------------------------------------------------------+
|class|text                                                                                                                                                       |
+-----+-----------------------------------------------------------------------------------------------------------------------------------------------------------+
|ham  |Go until jurong point, crazy.. Available only in bugis n great world la e buffet... Cine there got amore wat...                                            |
|ham  |Ok lar... Joking wif u oni...                                                                                                                              |
|spam |Free entry in 2 a wkly comp to win FA Cup final tkts 21st May 2005. Text FA to 87121 to receive entry question(std txt rate)T&C's apply 08452810075over18's|
|ham  |U dun say

In [10]:
from pyspark.sql.functions import length

In [11]:
data = data.withColumn('length', length(data.text))

In [13]:
data.show(5)

+-----+--------------------+------+
|class|                text|length|
+-----+--------------------+------+
|  ham|Go until jurong p...|   111|
|  ham|Ok lar... Joking ...|    29|
| spam|Free entry in 2 a...|   155|
|  ham|U dun say so earl...|    49|
|  ham|Nah I don't think...|    61|
+-----+--------------------+------+
only showing top 5 rows



In [14]:
data.groupBy('class').mean().show()

+-----+-----------------+
|class|      avg(length)|
+-----+-----------------+
|  ham|71.45431945307645|
| spam|138.6706827309237|
+-----+-----------------+



In [18]:
from pyspark.ml.feature import (Tokenizer, StopWordsRemover, 
                                CountVectorizer, IDF, StringIndexer, 
                                VectorAssembler)

In [16]:
tokenizer = Tokenizer(inputCol= 'text', outputCol= 'token_text')
stop_remove = StopWordsRemover(inputCol= 'token_text', outputCol= 'stop_token')
count_vec = CountVectorizer(inputCol= 'stop_token', outputCol= 'c_vec')
idf = IDF(inputCol= 'c_vec', outputCol= 'tf_idf')

In [17]:
ham_spam_label_encoder = StringIndexer(inputCol='class', outputCol= 'label')

In [19]:
clean_up = VectorAssembler(inputCols= ['tf_idf', 'length'], outputCol= 'features')

In [20]:
from pyspark.ml.classification import NaiveBayes

In [21]:
nb = NaiveBayes()

In [22]:
from pyspark.ml import Pipeline

In [24]:
data_prep_pipe = Pipeline(stages= [ham_spam_label_encoder, 
                                   tokenizer, 
                                   stop_remove, 
                                   count_vec, 
                                   idf, clean_up])

In [25]:
cleaner = data_prep_pipe.fit(data)

In [26]:
clean_data = cleaner.transform(data)

In [27]:
print(clean_data.columns)

['class', 'text', 'length', 'label', 'token_text', 'stop_token', 'c_vec', 'tf_idf', 'features']


In [28]:
clean_data = clean_data.select('label', 'features')

In [33]:
clean_data.show(5)

+-----+--------------------+
|label|            features|
+-----+--------------------+
|  0.0|(13424,[7,11,31,6...|
|  0.0|(13424,[0,24,297,...|
|  1.0|(13424,[2,13,19,3...|
|  0.0|(13424,[0,70,80,1...|
|  0.0|(13424,[36,134,31...|
+-----+--------------------+
only showing top 5 rows



In [36]:
xTrain, xTest = clean_data.randomSplit([0.7, 0.3], seed= 42)

In [37]:
spam_detector = nb.fit(xTrain)

In [38]:
data.printSchema()

root
 |-- class: string (nullable = true)
 |-- text: string (nullable = true)
 |-- length: integer (nullable = true)



In [39]:
test_result = spam_detector.transform(xTest)

In [41]:
test_result.show(5)

+-----+--------------------+--------------------+--------------------+----------+
|label|            features|       rawPrediction|         probability|prediction|
+-----+--------------------+--------------------+--------------------+----------+
|  0.0|(13424,[0,1,2,41,...|[-1060.7325420854...|[1.0,9.6391158107...|       0.0|
|  0.0|(13424,[0,1,5,20,...|[-803.13623340156...|[1.0,2.7071860143...|       0.0|
|  0.0|(13424,[0,1,7,8,1...|[-1152.0926413349...|[1.0,6.3682506790...|       0.0|
|  0.0|(13424,[0,1,7,15,...|[-656.71821333935...|[1.0,7.6641099247...|       0.0|
|  0.0|(13424,[0,1,12,33...|[-444.22584589378...|[1.0,1.4534997554...|       0.0|
+-----+--------------------+--------------------+--------------------+----------+
only showing top 5 rows



In [42]:
print(test_result.columns)

['label', 'features', 'rawPrediction', 'probability', 'prediction']


In [43]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

In [44]:
evaluator = MulticlassClassificationEvaluator(labelCol= 'label', 
                                              predictionCol= 'prediction', 
                                              metricName= 'accuracy')

In [46]:
accuracy = evaluator.evaluate(test_result)

In [51]:
print(accuracy)

0.9196484620213434


In [48]:
acc_eval = MulticlassClassificationEvaluator()

In [49]:
acc = acc_eval.evaluate(test_result)

In [50]:
print(acc)

0.9266021977210805
